In [1]:
import numpy as np
import pandas as pd

#after testing replace StringIO(temp) to filename
df = pd.read_csv("../input/cps_00054.csv")

In [2]:
# Sample Selection - HPV Sample C
# --------------------------------

df["YEAR"] = df["year".upper()] - 1 # income questions ask about 'last year'
df["AGE"]  = df["AGE".upper()]  - 1 # income questions ask about 'last year'

# Prime age
df = df[df["AGE"]>=25] # Drop if if age < 25
df = df[df["AGE"]<=60] # drop if age > 60

df = df[df["YEAR"]>=25]
# Employed at firm
df = df[ (df['CLASSWLY'] == 22) ]  # drop self-employed, government workers
     
# Drop if missing values    
df = df.dropna(subset=["OCCLY", "IND90LY", "AGE", 'CLASSWLY', 'RACE', 'INCWAGE', 'WKSWORK1', 'UHRSWORKLY', 'EDUC', "YEAR"])
df = df.rename(columns={"OCC90LY":"original"})
df = df[df["INCWAGE"] < 99999998] # Topcoded income

In [3]:
# Separate by OCC Code system - AW attempt
# https://cps.ipums.org/cps-action/variables/OCCLY#codes_section
# extend three years because it seems to work fine.
df80 = df[df['YEAR'].between(1983, 1991)]
df90 = df[df['YEAR'].between(1992, 2002)]
df00 = df[df['YEAR'].between(2003, 2010)]

# Load xwalks from Deming

xwalk80 = pd.read_stata("./new occ1990 codes/occ1980_occ1990dd.dta")
xwalk90 = pd.read_stata("./new occ1990 codes/occ1990_occ1990dd.dta")
xwalk00 = pd.read_stata("./new occ1990 codes/occ2000_occ1990dd.dta")

xwalk80 = xwalk80.rename(columns={"occ":"OCCLY"})
xwalk90 = xwalk90.rename(columns={"occ":"OCCLY"})
xwalk00 = xwalk00.rename(columns={"occ":"OCCLY"})
# Adjust so merges with CPS
xwalk00['OCCLY'] = xwalk00['OCCLY'] * 10

# merge in occ1990dd

df80 = pd.merge(df80,xwalk80, on='OCCLY', validate="m:1",how='left')
print("Num. of OCC80 codes without occ1990dd = ", np.sum(df80['occ1990dd'].isnull()))
print("Number of OCC80 codes with occ1990dd  = ", np.sum(df80['occ1990dd'].notnull()))
print("--")

df90 = pd.merge(df90,xwalk90, on='OCCLY', validate="m:1",how='left')
print("Num. of OCC90 codes without occ1990dd = ", np.sum(df90['occ1990dd'].isnull()))
print("Number of OCC90 codes with occ1990dd  = ", np.sum(df90['occ1990dd'].notnull()))
print("--")

df00 = pd.merge(df00,xwalk00, on='OCCLY', validate="m:1",how='left')
print("Num. of OCC00 codes without occ1990dd = ", np.sum(df00['occ1990dd'].isnull()))
print("Number of OCC00 codes with occ1990dd  = ", np.sum(df00['occ1990dd'].notnull()))
print("--")

# Check those occupation codes which do not merge with occ1990dd

df80['OCCLY'][df80['occ1990dd'].isnull()].value_counts() # These codes do note appear in the CPS list
df90['OCCLY'][df90['occ1990dd'].isnull()].value_counts() # These codes do note appear in the CPS list
df00['OCCLY'][df00['occ1990dd'].isnull()].sort_values().value_counts() # These codes do note appear in the CPS list


# Append the dataframes back together
cf = pd.concat([df80, df90, df00], ignore_index=True)


cf = cf[cf['occ1990dd'].notnull()]# drop if missing occ1990dd
cf['YEAR'].value_counts().sort_values()



Num. of OCC80 codes without occ1990dd =  3099
Number of OCC80 codes with occ1990dd  =  366948
--
Num. of OCC90 codes without occ1990dd =  62752
Number of OCC90 codes with occ1990dd  =  446483
--
Num. of OCC00 codes without occ1990dd =  7331
Number of OCC00 codes with occ1990dd  =  468947
--


1995    37046
1996    37835
1997    38470
1983    38587
1988    38952
1998    39115
1999    39802
1991    40092
1984    40270
1985    40512
1986    40735
1993    41166
1987    41286
1994    41762
1992    42764
1989    43067
1990    43447
2010    50704
2009    58872
2006    59412
2007    59417
2005    59569
2004    59793
2008    59990
2003    61190
2001    64057
2000    64466
Name: YEAR, dtype: int64

In [4]:


df=cf
 


In [5]:
# Variables
# --------------------------------

# Sex
df["FEMALE"] = df["SEX"].replace(1, 0)
df["FEMALE"] = df["FEMALE"].replace(2, 1)
df["MALE"] = df["SEX"].replace(2  , 0)

# Big firm has 1000+ employees (=9, 500-999 = 8)
df["big_firm"]  = df["firmsize".upper()]
for x in range(9):
    df["big_firm"] = df["big_firm"].replace(x,0)

df["big_firm"] = df["big_firm"].replace(9,1)
df['big_firm'].fillna(0, inplace =True)

# Race 
for x in df["RACE"].unique():
    if x == 100:
        df["RACE"] = df["RACE"].replace(x,1)
    else:
        df["RACE"] = df["RACE"].replace(x,0)
        
        

# Occupation
df['2digit occupation'] = pd.cut(x=df['occ1990dd'], bins=[0,22, 37,200, 235,283,389,408,427,444,447,455,472,498,549,599,617,699,799,890], labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], right=False)
df["1digit occupation"] = pd.cut(x=df["occ1990dd"], bins=[0,199,389,472,498,699,889], labels=[1,2,3,4,5,6])
# Industry
df["1digit industry"]   = pd.cut(x=df["IND90LY"], bins=[0,200,391, 473, 498, 890,1000], labels=[2,3,4,5,1,6])

# Education 
kf = pd.read_stata("../input/educ_years_school.dta")

if "years_school" in df:
    df = df.drop(columns=["years_school"])
    
educ_df = kf.rename(columns={"educ":"EDUC"})
df = pd.merge(df, educ_df, on='EDUC', how='right') # merge in years of schooling
df = df.rename(columns={"years_school_y":"years_school"})

# Experience
df["exp"]  = (df["AGE"] - np.maximum(df["years_school"], 12)) - 6
df["exp2"] = (df["exp"] ** 2 )
df = df[df["exp"] >= 0] 


# Hours
df["annual_hours"] = df["WKSWORK1"] * df["UHRSWORKLY"]
df = df[df["annual_hours"]>=260] # drop if worked less than a month of 8hr days

# Drop if report income, but no hours.
indexNames = df[(df['INCWAGE'] > 0) & (df['annual_hours'] == 0)].index
df.drop(indexNames , inplace=True)


# Earnings
df['inc_self'] = 0 # df['OINCBUS'] + df['OINCFARM']    
df['earnings'] = (2/3) * df['inc_self'] + df['INCWAGE'] # 2/3 business income + labor income


In [6]:
# Adjust for inflation
# --------------------------------

df = df.rename(columns={"YEAR":"year"})

#Merge in cpi data
cpi_df = pd.read_csv("../output/revised_CPI.csv") 
df     = pd.merge(df, cpi_df, on='year', how='right')

# Merge in fedminwage data
fedminwage_df = pd.read_csv("../input/fedminwage.csv") 
df = pd.merge(df, fedminwage_df, on='year', how='right') # real_cpis is cpi / cpi[2000]
df["earnings1"] = df["earnings"]
# Normalize to 2000-dollars
df["earnings"]   = df["earnings"] / df["real_cpis"]
df["fedminwage"] = df["fedminwage"] / df["real_cpis"]

# Compute implied hourly wage
df["Wage"] = df["earnings"] / df["annual_hours"]

# Drop if wage is less than 1/2 fedminwage
df = df.query("fedminwage * .5 <= Wage")# same thing: df[(df['Wage'] > df["fedminwage"])]
#df = df[df["Wage"]>= .5 * df["fedminwage"]]

In [7]:
# Clean up
# --------------------------------
df = df.rename(columns={'occ1990dd': "3digit occupation", "RACE":"White","EDUC":"education", "annual_hours":"Annual hours"})

k = {}
for x in df.columns:
    k[x] = x.capitalize()
df = df.rename(columns=k)
columnsTitles = ['Year', '3digit occupation', '2digit occupation', '1digit occupation', 'Age', 'Asecwt', 'Exp','Male', 'Female','White', 'Exp2','1digit industry', 'Big_firm', "Education","Annual hours","Wage", "Years_school"]
df = df.reindex(columns=columnsTitles)
df = df.reset_index(drop=True)

df[df.columns[:-1]] = df[df.columns[:-1]].astype('int64')
df[["Wage"]] = df["Wage"].astype('float64')
df["Wage"]   = df["Wage"].round(2)

# Save
df.to_csv("../output/current_values.csv")

In [8]:
df.head()

,Year,3digit occupation,2digit occupation,1digit occupation,Age,Asecwt,Exp,Male,Female,White,Exp2,1digit industry,Big_firm,Education,Annual hours,Wage,Years_school
0,1983,754,18,6,42,568,24,1,0,1,576,2,0,40,2080,13.0,9.0
1,1983,745,18,6,52,575,34,0,1,1,1156,3,0,40,1960,7.0,9.0
2,1983,745,18,6,50,586,32,1,0,1,1024,3,0,40,2080,15.0,9.0
3,1983,727,18,6,31,603,13,1,0,1,169,3,0,40,1980,11.0,9.0
4,1983,734,18,6,30,688,12,1,0,1,144,3,0,40,2208,7.0,9.0
